# `__init__`

In [1]:
import summaflow as sf
import geopandas as gpd
import xarray as xr
import pint_xarray
import pint_pandas
import pint

import os

In [2]:
sf.__file__

'/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/__init__.py'

In [3]:
# paths
root_path = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path, 'bcalgary_subbasins.shp')

riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

In [4]:
exp = sf.SUMMAWorkflow(
    forcing_data = ['1', '2'],
    forcing_vars = {
        'pr': 'precipitation',
        'temp': 'airtemp',
        'measurement_height': 'mHeight',
    },
    forcing_units = {
        'pr': 'mm',
        'temp': 'degC',
    },
    forcing_to_units = {
        'pr': 'kgm^/s',
        'temp': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_units = {'a': 'm', 'c': 'ha'},
    topology_to_units = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
)

In [5]:
exp

Forcing files: ['1', '2']
GRUs: 169
HRUs: 169
Rivers: 169 
Initialized: False

In [6]:
exp.init_attrs()

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/utils.py:192: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


In [ ]:
exp.attrs